In [3]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime

# URL e mapeamento de status
STATUS_URL = "https://www.viamobilidade.com.br/"
statuses = {
    "green": "Operação Normal",
    "yellow": "Operação Parcial",
    "red": "Operação Interrompida"
}

# Função para buscar o status das linhas
def get_status():
    try:
        response = requests.get(STATUS_URL, timeout=5)
        response.raise_for_status()
        html = response.text
        soup = BeautifulSoup(html, "html.parser")
        lines = []

        for line in soup.select("li[class*='line-']"):
            name = line.select_one("span[title]").get("title") if line.select_one("span[title]") else None
            line_number = next((c.split('-')[1] for c in line.get("class", []) if c.startswith("line-")), None)
            reason = line.select_one("p").text if line.select_one("p") else None
            status_element = line.select_one(".status")
            status_description = status_element.text if status_element else None
            status_color = next((c for c in status_element.get("class", []) if c != "status"), None) if status_element else None
            status = statuses.get(status_color)

            lines.append({
                "name": name,
                "number": line_number,
                "reason": reason,
                "statusDescription": status_description,
                "status": status
            })

        raw_date = soup.select_one(".lines p strong").text if soup.select_one(".lines p strong") else None
        updated_at = datetime.strptime(raw_date, "%d/%m/%Y %H:%M:%S").isoformat() if raw_date else None

        return {
            "lines": lines,
            "updatedAt": updated_at
        }
    except requests.RequestException as e:
        print(f"Erro ao obter o status: {e}")
        return {
            "lines": [],
            "updatedAt": None
        }

# Testando a função
if __name__ == "__main__":
    status_data = get_status()
    print(status_data)

{'lines': [{'name': 'Linha 1 - Azul', 'number': '1', 'reason': 'Operação Normal', 'statusDescription': 'Operação Normal', 'status': None}, {'name': 'Linha 2 - Verde', 'number': '2', 'reason': 'Operação Normal', 'statusDescription': 'Operação Normal', 'status': None}, {'name': 'Linha 3 - Vermelha', 'number': '3', 'reason': 'Operação Normal', 'statusDescription': 'Operação Normal', 'status': None}, {'name': 'Linha 4-Amarela', 'number': '4', 'reason': 'Devido às obras de modernização na estação Palmeiras-Barra Funda, passageiros com destino à estação Júlio Prestes devem embarcar no trem da plataforma 4.', 'statusDescription': 'Operação Normal', 'status': None}, {'name': 'Linha 5-Lilás', 'number': '5', 'reason': 'Devido às obras de modernização na estação Palmeiras-Barra Funda, passageiros com destino à estação Júlio Prestes devem embarcar no trem da plataforma 4.', 'statusDescription': 'Operação Normal', 'status': None}, {'name': 'RUBI', 'number': '7', 'reason': 'Devido às obras de modern

In [6]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime

# URL e mapeamento de status
STATUS_URL = "https://www.viamobilidade.com.br/"
statuses = {
    "green": "Operação Normal",
    "yellow": "Operação Parcial",
    "red": "Operação Interrompida"
}

# Função para buscar o status das linhas
def get_status():
    try:
        response = requests.get(STATUS_URL, timeout=5)
        response.raise_for_status()
        html = response.text
        soup = BeautifulSoup(html, "html.parser")
        lines = []

        for line in soup.select("li[class*='line-']"):
            name = line.select_one("span[title]").get("title") if line.select_one("span[title]") else None
            line_number = next((c.split('-')[1] for c in line.get("class", []) if c.startswith("line-")), None)
            reason = line.select_one("p").text if line.select_one("p") else None
            status_element = line.select_one(".status")
            status_description = status_element.text if status_element else None
            status_color = next((c for c in status_element.get("class", []) if c != "status"), None) if status_element else None
            status = statuses.get(status_color)

            lines.append({
                "name": name,
                "number": line_number,
                "reason": reason,
                "statusDescription": status_description,
                "status": status
            })

        raw_date = soup.select_one(".lines p strong").text if soup.select_one(".lines p strong") else None
        updated_at = datetime.strptime(raw_date, "%d/%m/%Y %H:%M:%S").isoformat() if raw_date else None

        return {
            "lines": lines,
            "updatedAt": updated_at
        }
    except requests.RequestException as e:
        print(f"Erro ao obter o status: {e}")
        return {
            "lines": [],
            "updatedAt": None
        }

# Handlers da Skill
class CheckStatusIntentHandler:
    def handle(self, handler_input):
        slots = handler_input.request_envelope["request"]["intent"]["slots"]
        line_name = slots["line"]["value"] if "line" in slots else None

        # Validação do slot
        if not line_name:
            return "Desculpe, não consegui entender qual linha você mencionou. Por favor, repita."

        status_data = get_status()
        if status_data["lines"]:
            line_status = next((line for line in status_data["lines"] if line["name"] and line_name.lower() in line["name"].lower()), None)
            if line_status:
                speech_text = f"A linha {line_status['name']} está com status de {line_status['status']}."
                if line_status["reason"]:
                    speech_text += f" Motivo: {line_status['reason']}."
                if status_data["updatedAt"]:
                    speech_text += f" Última atualização foi em {datetime.fromisoformat(status_data['updatedAt']).strftime('%d/%m/%Y %H:%M:%S')}."
            else:
                speech_text = f"Não consegui encontrar informações para a linha {line_name}."
        else:
            speech_text = "Não consegui encontrar o status das linhas no momento."

        return speech_text

# Mock para simular o handler input
class MockHandlerInput:
    def __init__(self, request_type, slots=None):
        self.request_envelope = {
            "request": {
                "type": request_type,
                "intent": {
                    "slots": slots if slots else {}
                }
            }
        }




# Testando o CheckStatusIntentHandler
if __name__ == "__main__":
    # Simulando uma entrada para checar o status da linha "Verde"
    handler_input = MockHandlerInput("IntentRequest", {"line": {"value": "Verde"}})
    check_status_handler = CheckStatusIntentHandler()
    response = check_status_handler.handle(handler_input)
    print(response)

A linha Linha 2 - Verde está com status de None. Motivo: Operação Normal. Última atualização foi em 01/10/2024 18:38:11.


In [7]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime

# URL e mapeamento de status
STATUS_URL = "https://www.viamobilidade.com.br/"
statuses = {
    "green": "Operação Normal",
    "yellow": "Operação Parcial",
    "red": "Operação Interrompida"
}

# Função para buscar o status das linhas
def get_status():
    try:
        response = requests.get(STATUS_URL, timeout=5)
        response.raise_for_status()
        html = response.text
        soup = BeautifulSoup(html, "html.parser")
        lines = []

        for line in soup.select("li[class*='line-']"):
            name = line.select_one("span[title]").get("title") if line.select_one("span[title]") else None
            line_number = next((c.split('-')[1] for c in line.get("class", []) if c.startswith("line-")), None)
            reason = line.select_one("p").text if line.select_one("p") else None
            status_element = line.select_one(".status")
            status_description = status_element.text if status_element else None
            status_color = next((c for c in status_element.get("class", []) if c != "status"), None) if status_element else None
            status = statuses.get(status_color)

            lines.append({
                "name": name,
                "number": line_number,
                "reason": reason,
                "statusDescription": status_description,
                "status": status
            })

        raw_date = soup.select_one(".lines p strong").text if soup.select_one(".lines p strong") else None
        updated_at = datetime.strptime(raw_date, "%d/%m/%Y %H:%M:%S").isoformat() if raw_date else None

        return {
            "lines": lines,
            "updatedAt": updated_at
        }
    except requests.RequestException as e:
        print(f"Erro ao obter o status: {e}")
        return {
            "lines": [],
            "updatedAt": None
        }

# Handlers da Skill
class CheckStatusIntentHandler:
    def handle(self, handler_input):
        slots = handler_input.request_envelope["request"]["intent"]["slots"]
        line_name = slots["line"]["value"] if "line" in slots else None

        # Validação do slot
        if not line_name:
            return "Desculpe, não consegui entender qual linha você mencionou. Por favor, repita."

        status_data = get_status()
        if status_data["lines"]:
            line_status = next((line for line in status_data["lines"] if line["name"] and line_name.lower() in line["name"].lower()), None)
            if line_status:
                speech_text = f"A linha {line_status['name']} está com status de {line_status['status']}."
                if line_status["reason"]:
                    speech_text += f" Motivo: {line_status['reason']}."
                if status_data["updatedAt"]:
                    speech_text += f" Última atualização foi em {datetime.fromisoformat(status_data['updatedAt']).strftime('%d/%m/%Y %H:%M:%S')}."
            else:
                speech_text = f"Não consegui encontrar informações para a linha {line_name}."
        else:
            speech_text = "Não consegui encontrar o status das linhas no momento."

        return speech_text

# Mock para simular o handler input
class MockHandlerInput:
    def __init__(self, request_type, slots=None):
        self.request_envelope = {
            "request": {
                "type": request_type,
                "intent": {
                    "slots": slots if slots else {}
                }
            }
        }


def get_status():
    try:
        response = requests.get(STATUS_URL, timeout=5)
        response.raise_for_status()
        html = response.text
        soup = BeautifulSoup(html, "html.parser")
        lines = []

        for line in soup.select("li[class*='line-']"):
            name = line.select_one("span[title]").get("title") if line.select_one("span[title]") else None
            line_number = next((c.split('-')[1] for c in line.get("class", []) if c.startswith("line-")), None)
            reason = line.select_one("p").text if line.select_one("p") else None
            status_element = line.select_one(".status")
            status_description = status_element.text if status_element else None
            status_color = next((c for c in status_element.get("class", []) if c != "status"), None) if status_element else None
            
            # Debug prints
            print(f"Line Name: {name}, Line Number: {line_number}, Reason: {reason}, Status Description: {status_description}, Status Color: {status_color}")

            status = statuses.get(status_color)
            lines.append({
                "name": name,
                "number": line_number,
                "reason": reason,
                "statusDescription": status_description,
                "status": status
            })

        raw_date = soup.select_one(".lines p strong").text if soup.select_one(".lines p strong") else None
        updated_at = datetime.strptime(raw_date, "%d/%m/%Y %H:%M:%S").isoformat() if raw_date else None

        return {
            "lines": lines,
            "updatedAt": updated_at
        }
    except requests.RequestException as e:
        print(f"Erro ao obter o status: {e}")
        return {
            "lines": [],
            "updatedAt": None
        }

# Testando o CheckStatusIntentHandler
if __name__ == "__main__":
    # Simulando uma entrada para checar o status da linha "Verde"
    handler_input = MockHandlerInput("IntentRequest", {"line": {"value": "Verde"}})
    check_status_handler = CheckStatusIntentHandler()
    response = check_status_handler.handle(handler_input)
    print(response)

Line Name: Linha 1 - Azul, Line Number: 1, Reason: Operação Normal, Status Description: Operação Normal, Status Color: verde
Line Name: Linha 2 - Verde, Line Number: 2, Reason: Operação Normal, Status Description: Operação Normal, Status Color: verde
Line Name: Linha 3 - Vermelha, Line Number: 3, Reason: Operação Normal, Status Description: Operação Normal, Status Color: verde
Line Name: Linha 4-Amarela, Line Number: 4, Reason: Devido às obras de modernização na estação Palmeiras-Barra Funda, passageiros com destino à estação Júlio Prestes devem embarcar no trem da plataforma 4., Status Description: Operação Normal, Status Color: verde
Line Name: Linha 5-Lilás, Line Number: 5, Reason: Devido às obras de modernização na estação Palmeiras-Barra Funda, passageiros com destino à estação Júlio Prestes devem embarcar no trem da plataforma 4., Status Description: Operação Normal, Status Color: verde
Line Name: RUBI, Line Number: 7, Reason: Devido às obras de modernização na estação Palmeiras-